<a href="https://colab.research.google.com/github/vedanshtyagi1901/deocker-demo/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Install Kaggle CLI if not already
!pip install -q kaggle

# Download dataset
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

# Unzip
!unzip imdb-dataset-of-50k-movie-reviews.zip

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 1.03GB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [7]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [8]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [10]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [11]:
data["sentiment"].value_counts()


,count
sentiment,
1,25000
0,25000


In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)


In [14]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [15]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [16]:
model = Sequential()
model.add(Embedding(input_dim =5000, output_dim = 128, input_length = 200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = "sigmoid"))

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [21]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [22]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2)


Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 205s 393ms/step - accuracy: 0.7158 - loss: 0.5409 - val_accuracy: 0.8524 - val_loss: 0.3510
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 194s 386ms/step - accuracy: 0.8550 - loss: 0.3452 - val_accuracy: 0.8729 - val_loss: 0.3224
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 193s 387ms/step - accuracy: 0.8819 - loss: 0.2926 - val_accuracy: 0.8411 - val_loss: 0.3791
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 195s 389ms/step - accuracy: 0.8699 - loss: 0.3116 - val_accuracy: 0.8702 - val_loss: 0.3197
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 389ms/step - accuracy: 0.9044 - loss: 0.2454 - val_accuracy: 0.8752 - val_loss: 0.3108


In [23]:
model.save("model.h5")


In [24]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [25]:
loss, accuracy = model.evaluate(X_test, Y_test)


313/313 ━━━━━━━━━━━━━━━━━━━━ 34s 107ms/step - accuracy: 0.8758 - loss: 0.2986


In [26]:
print(loss)


0.292378306388855


In [27]:
print(accuracy)


0.8813999891281128


In [28]:
print(tf.__version__)

NameError: name 'tf' is not defined

In [30]:
import tensorflow as tf
print(tf.__version__)


2.18.0
